In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Head_and_Neck_Cancer/GSE201777'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Differentially expressed genes related to lymph node metastasis in advanced laryngeal squamous cell cancers"
!Series_summary	"Understanding the molecular mechanisms and gene expression in laryngeal squamous cell carcinoma (LSCC) may explain its aggressive biological behavior and regional metastasis pathways. Better understanding of the molecular mechanisms underlying LSCC metastasis and the search for possible molecular targets seems promising. Interpreting the links between the differentially expressed genes in advanced stages can lead to a search for predictive markers that can also help determine the possible treatment routes. We designed this study to detect possible genetic alterations in a homogeneous group of patients with locoregionally advanced laryngeal cancer who underwent total laryngectomy and neck dissection. Patients with and without lymph node metastasis were selected to examine the differential gene expression in the normal mucosa

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Checking if gene expression data is available
is_gene_available = True  # Assuming we found gene expression data

# Identifying keys for the required variables from sample characteristics dictionary
sample_characteristics_dict = {
    0: ['patient diagnosis: laryngeal squamous cell carcinoma (LSCC)'], 
    1: ['tissue: Lymph node', 'tissue: Tumor', 'tissue: Mucosa'], 
    2: ['lymph node metastasis: negative', 'lymph node metastasis: positive', 'lymph node metastasis: positiive']
}

# For simplicity, let's assume that the trait "Head_and_Neck_Cancer" corresponds to metastasis status
trait_row = 2  # Since lymph node metastasis indicates advanced cancer

# No information about age and gender found
age_row = gender_row = None

# Function to convert trait values
def convert_trait(value):
    status = value.split(':')[-1].strip().lower()
    if 'positive' in status:
        return 1
    elif 'negative' in status:
        return 0
    else:
        return None

# No conversion needed for age and gender as they are not available
convert_age = None
convert_gender = None

# Saving cohort information
save_cohort_info('GSE201777', './preprocessed/Head_and_Neck_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Extracting clinical features if applicable
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Head_and_Neck_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Head_and_Neck_Cancer/trait_data/GSE201777.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM6071161': [0], 'GSM6071162': [0], 'GSM6071163': [0], 'GSM6071164': [0], 'GSM6071165': [0], 'GSM6071166': [0], 'GSM6071167': [0], 'GSM6071168': [0], 'GSM6071169': [0], 'GSM6071170': [1], 'GSM6071171': [None], 'GSM6071172': [1], 'GSM6071173': [1], 'GSM6071174': [1], 'GSM6071175': [None], 'GSM6071176': [0], 'GSM6071177': [0], 'GSM6071178': [0], 'GSM6071179': [0], 'GSM6071180': [0], 'GSM6071181': [0], 'GSM6071182': [0], 'GSM6071183': [0], 'GSM6071184': [0], 'GSM6071185': [0], 'GSM6071186': [0], 'GSM6071187': [0], 'GSM6071188': [None], 'GSM6071189': [1], 'GSM6071190': [None], 'GSM6071191': [1], 'GSM6071192': [1], 'GSM6071193': [None], 'GSM6071194': [1], 'GSM6071195': [1], 'GSM6071196': [1], 'GSM6071197': [1], 'GSM6071198': [None], 'GSM6071199': [0], 'GSM6071200': [0], 'GSM6071201': [0], 'GSM6071202': [1], 'GSM6071203': [None], 'GSM6071204': [1], 'GSM6071205': [1], 'GSM6071206': [None], 'GSM6071207': [1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at',
       '11715104_s_at', '11715105_at', '11715106_x_at', '11715107_s_at',
       '11715108_x_at', '11715109_at', '11715110_at', '11715111_s_at',
       '11715112_at', '11715113_x_at', '11715114_x_at', '11715115_s_at',
       '11715116_s_at', '11715117_x_at', '11715118_s_at', '11715119_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'GeneChip Array': ['Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array'], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['30-Mar-16', '30-Mar-16', '30-Mar-16', '30-Mar-16', '30-Mar-16'], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database'], 'Transcript ID(Array Design)': ['g21264570', 'g21264570', 'g21264570', 'g22748780', 'g30039713'], 'Target Description': ['g21264570 /TID=g21264570 /CNT=1 /FEA=FLmRNA /TIER=FL /STK=0 /

### Step 6: Gene Identifier Mapping

In [7]:

# 1. Determine the identifier key and gene symbol key from the gene annotation dictionary
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Head_and_Neck_Cancer/gene_data/GSE201777.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Head_and_Neck_Cancer')

# 4. Save the cohort information.
save_cohort_info('GSE201777', './preprocessed/Head_and_Neck_Cancer/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Head_and_Neck_Cancer/GSE201777.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Head_and_Neck_Cancer', the least common label is '1.0' with 15 occurrences. This represents 38.46% of the dataset.
The distribution of the feature 'Head_and_Neck_Cancer' in this dataset is fine.

A new JSON file was created at: ./preprocessed/Head_and_Neck_Cancer/cohort_info.json
